In [8]:
k = 0
for i in range(0, 8):
    for j in range(0, 8):
        print("{0:06b}".format(k), end=" ")
        k += 1
    print("")
print("")

000000 000001 000010 000011 000100 000101 000110 000111 
001000 001001 001010 001011 001100 001101 001110 001111 
010000 010001 010010 010011 010100 010101 010110 010111 
011000 011001 011010 011011 011100 011101 011110 011111 
100000 100001 100010 100011 100100 100101 100110 100111 
101000 101001 101010 101011 101100 101101 101110 101111 
110000 110001 110010 110011 110100 110101 110110 110111 
111000 111001 111010 111011 111100 111101 111110 111111 



In [30]:
import math
a = 4 # Starts from 0
i = 0 # Starts from 0
p = 8
pi = (a + (-1)**(a % 2)*(-1)**(i % 2)*2**math.floor(i/2)) % p
pialt = (a + -1**(a+i)*2**math.floor(i/2)) % p
print(pi)
print(pialt)

5
3


# 1D Version

In [91]:
import copy
p = 128
recv_data = []
sum_of_distances = 0
contagion = []
bw_new = 0
bw_ref = 0
bw_ideal = 0
for i in range(0, p):
    recv_data += [[i]]
    contagion += (0, 0)
for i in range(0, int(math.log2(p))):
    recv_data_new = copy.deepcopy(recv_data)
    distances = [1, 1, 3, 5, 11, 21, 43] # Jacobsthal sequence!! (https://oeis.org/search?q=1%2C+1%2C+3%2C+5%2C+11%2C+21&language=english&go=Search)
    for j in range(0, p):       
        distance = distances[i]
        if j % 2 == 0 and i % 2 == 0: # Even id, even iteration (starting form 0)
            peer = (j+distance) % p
        elif j % 2 == 0 and i % 2 != 0: # Even id, odd iteration
            peer = (j-distance) % p
        elif j % 2 != 0 and i % 2 == 0: # Odd id, even iteration
            peer = (j-distance) % p
        else: # Odd id, odd iteration
            peer = (j+distance) % p
        sum_of_distances += distance 
        recv_data_new[j].extend(recv_data[peer])
        if j == 0:
            bw_ideal += (1/2**(i+1)) 
            bw_new += (1/2**(i+1))*distance
            bw_ref += (1/2**(i+1))*2**i
    recv_data = copy.deepcopy(recv_data_new)

first = recv_data[0]
for i in range(0, p):
    if sorted(first) != sorted(recv_data[i]):
        print("FAIL!")
        break        
#print(recv_data)
print("Sum of distances: " + str(sum_of_distances / p) + " ref: " + str(p-1))
print("Bw new: " + str(bw_new) + " bw ref: " + str(bw_ref) + " bw ideal: " + str(bw_ideal))


FAIL!
Sum of distances: 383.0 ref: 127
Bw new: 4.9453125 bw ref: 3.5 bw ideal: 0.9921875


# 2D Version

In [17]:
import math
import copy
import random
def get_node_at_distance_in_direction(n, d, dir, rows, cols):
    (r, c) = n // rows, n % cols
    if dir == 'E':
        dest_coord = (r, (c+d) % cols)
    elif dir == 'S':
        dest_coord = ((r+d) % cols, c)
    elif dir == 'W':
        dest_coord = (r, (c-d) % cols)
    else: # 'N'
        dest_coord = ((r-d) % cols, c)
    r_d, c_d = dest_coord
    return (r_d)*cols + c_d

rows = 32
cols = 32
p = rows*cols
recv_data = []
sum_of_distances = 0
sum_of_distances_ref = 0
bw_new = 0
bw_ref = 0
bw_ideal = 0
directions_even_even = ['E', 'S', 'W', 'N']
directions_even_odd = ['W', 'S', 'E', 'N']
directions_odd_even = ['E', 'N', 'W', 'S']
directions_odd_odd = ['W', 'N', 'E', 'S']
for i in range(0, p):
    recv_data += [[i]]
for i in range(0, int(math.log2(p))):
    recv_data_new = copy.deepcopy(recv_data)
    distances = [1, 1, 1, 1, 5, 9, 17, 33] # Pentanacci numbers
    distances = [1, 1, 1, 1, 3, 3, 5, 5, 11, 11, 21, 21, 43, 43] # WTF! (it is working!?!)
    for j in range(0, p):       
        distance = distances[i]
        row = j // rows
        col = j % cols
        if row % 2 == 0 and col % 2 == 0: # Even row, even col
            peer = get_node_at_distance_in_direction(j, distances[i], directions_even_even[i % 4], rows, cols)        
        elif row % 2 == 0 and col % 2 != 0: # Even row, odd col
            peer = get_node_at_distance_in_direction(j, distances[i], directions_even_odd[i % 4], rows, cols)
        elif row % 2 != 0 and col % 2 == 0: # Odd row, even col
            peer = get_node_at_distance_in_direction(j, distances[i], directions_odd_even[i % 4], rows, cols)
        else: # Odd row, odd col
            peer = get_node_at_distance_in_direction(j, distances[i], directions_odd_odd[i % 4], rows, cols)        
        recv_data_new[j].extend(recv_data[peer])
        if j == 0:
            sum_of_distances += distance 
            sum_of_distances_ref += 2**math.floor(i/2)
            bw_ideal += (1/2**(i+1)) 
            bw_new += (1/2**(i+1))*distance
            bw_ref += (1/2**(i+1))*2**math.floor(i/2)
    recv_data = copy.deepcopy(recv_data_new)

first = recv_data[0]
for i in range(0, p):
    if sorted(first) != sorted(recv_data[i]):
        print("FAIL!")
        break        
#print(recv_data)
print("Sum of distances: " + str(sum_of_distances) + " ref: " + str(sum_of_distances_ref))
print("Bw new: " + str(bw_new) + " bw ref: " + str(bw_ref) + " bw ideal: " + str(bw_ideal))


[[49, 26, 2, 46, 41, 75, 74, 84, 5, 45, 94, 23, 41, 33, 3, 88, 90, 76, 27, 80, 61, 100, 97, 54, 97, 19, 93, 26, 99, 78, 54, 21, 33, 42, 74, 21, 41, 5, 67, 76, 81, 65, 78, 93, 98, 93, 19, 57, 52, 94, 54, 86, 62, 85, 95, 22, 8, 43, 17, 41, 7, 25, 34, 23, 18, 93, 46, 25, 95, 51, 24, 36, 68, 12, 69, 49, 57, 9, 67, 55, 26, 28, 88, 68, 39, 90, 70, 75, 36, 43, 96, 42, 68, 72, 21, 87, 31, 67, 77, 90, 16, 46, 93, 66, 19, 71, 74, 14, 71, 28, 52, 51, 61, 96, 96, 19, 68, 48, 53, 31, 37, 24, 35, 2, 14, 31, 59, 20, 48, 89, 23, 40, 9, 84, 40, 20, 21, 66, 84, 45, 31, 19, 13, 95, 51, 38, 70, 37, 60, 16, 62, 52, 1, 56, 47, 6, 3, 85, 58, 11, 20, 4, 93, 84, 75, 57, 59, 65, 21, 1, 41, 96, 21, 39, 85, 91, 74, 14, 57, 67, 60, 63, 39, 18, 3, 47, 53, 64, 33, 79, 89, 84, 24, 14, 28, 50, 51, 84, 44, 99, 30, 74, 28, 15, 87, 49, 88, 91, 35, 21, 57, 92, 83, 80, 24, 18, 23, 8, 28, 54, 51, 90, 8, 3, 83, 83, 61, 14, 19, 60, 79, 65, 34, 25, 78, 6, 61, 18, 82, 28, 35, 85, 68, 44, 18, 51, 93, 73, 4, 32, 8, 39, 58, 19, 23